In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoModel, AutoTokenizer
from experimental.t5_dataset import T5Dataset
import json
import os
import os.path as op

st_tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/msmarco-bert-base-dot-v5") #model_args={"embedding_size":512})
st_model = AutoModel.from_pretrained("sentence-transformers/msmarco-bert-base-dot-v5")#embedding_size=512)


t5_tokenizer = T5Tokenizer.from_pretrained("t5-base", extra_ids=1)


########Building Frankenstein's monster #########
t5_model = T5ForConditionalGeneration.from_pretrained("t5-base")

from experimental.test_t5 import STOnlyEncoder, CustomEncoder
t5_model.encoder = CustomEncoder(t5_model.encoder)
#t5_model.encoder = STOnlyEncoder(st_model, st_tokenizer=st_tokenizer, t5_tokenizer=t5_tokenizer) #Unfortunately all these parameters are necessary

######### Monster building complete ##########

#ORIGINAL_t5_model = T5ForConditionalGeneration.from_pretrained("t5-base")

DATA_DIR = "DialogSum_Data"

SAMPLE_DATA = op.join(DATA_DIR, 'dialogsum.sample.jsonl')
TEST_DATA = op.join(DATA_DIR, 'dialogsum.test.jsonl')
TRAIN_DATA = op.join(DATA_DIR, 'dialogsum.train.jsonl')
DEV_DATA = op.join(DATA_DIR, 'dialogsum.dev.jsonl')

def load_jsonl(filepath):
    
    output_list = []
    
    with open(filepath) as sd_file:
        lines = sd_file.readlines()
    
    for line in lines:
        output_list.append(json.loads(line))

    return output_list


if __name__ == "__main__":
    dev_data_list = load_jsonl(DEV_DATA)
    dev_dataset = T5Dataset(dev_data_list, 
                  tokenizer=t5_tokenizer,
                  debug=True)   
    for i in dev_dataset:
        print(i)


TypeError: __init__() missing 1 required positional argument: 'encoder'

In [ ]:
from IPython.display import clear_output

dev_dataset = T5Dataset(dev_data_list, 
                tokenizer=t5_tokenizer,
                debug=True)   

for i in dev_dataset:
    ids = i["source_ids"]
    mask = i["source_mask"]
   
    # print("#"*20)
    # print(ids)
    # print(mask)
    # print(dir(mask))
    # print(dir(t5_model))
   # print(type(ids))
   # print(type(mask))
    ORIG_encoding = t5_tokenizer.batch_encode_plus("some sentence here", padding=True, return_tensors="pt")
    ORIG_ids = ORIG_encoding["input_ids"]
    ORIG_mask = ORIG_encoding["attention_mask"]
    output = t5_model.generate(input_ids = ORIG_ids, attention_mask = ORIG_mask)
    # print(ORIG_output)
    # print("o"*10)
    # print(ORIG_ids.shape)
    print(ids.shape)
    # print("f"*10)
    # print(ORIG_mask.shape)
    print(mask.shape)

    # print()
    # print(ORIG_encoding)
    # break
    
    output = t5_model.generate(input_ids=ids, attention_mask = mask)
    print(t5_tokenizer.decode(output))
    break

torch.Size([178])
torch.Size([178])
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents: 1
n.sents

UnboundLocalError: local variable 'encoder_extended_attention_mask' referenced before assignment